# Modularnost

## Moduli

In [21]:
module Avtomat = struct

    type stanje = { oznaka : string }

    type avtomat = {
        stanja : stanje list;
        zacetno_stanje : stanje;
        sprejemna_stanja : stanje list;
        prehodi : (stanje * char * stanje) list;
    }

    let stanje oznaka = { oznaka }

    let prazen_avtomat zacetno_stanje =
    {
        stanja = [ zacetno_stanje ];
        zacetno_stanje;
        sprejemna_stanja = [];
        prehodi = [];
    }

    let dodaj_nesprejemno_stanje stanje avtomat =
    { avtomat with stanja = stanje :: avtomat.stanja }

    let dodaj_sprejemno_stanje stanje avtomat =
    {
        avtomat with
        stanja = stanje :: avtomat.stanja;
        sprejemna_stanja = stanje :: avtomat.sprejemna_stanja;
    }

    let dodaj_prehod stanje1 znak stanje2 avtomat =
    { avtomat with prehodi = (stanje1, znak, stanje2) :: avtomat.prehodi }

    let pomozna_funkcija_za_projekcijo_morebitnega_tretjega = function
    | None -> None
    | Some (_, _, q') -> Some q'


    let preberi_znak avtomat q znak =
        avtomat.prehodi
        |> List.find_opt (fun (q1, znak', _q2) -> q1 = q && znak = znak')
        |> pomozna_funkcija_za_projekcijo_morebitnega_tretjega

end

module Avtomat :
  sig
    type stanje = { oznaka : string; }
    type avtomat = {
      stanja : stanje list;
      zacetno_stanje : stanje;
      sprejemna_stanja : stanje list;
      prehodi : (stanje * char * stanje) list;
    }
    val stanje : string -> stanje
    val prazen_avtomat : stanje -> avtomat
    val dodaj_nesprejemno_stanje : stanje -> avtomat -> avtomat
    val dodaj_sprejemno_stanje : stanje -> avtomat -> avtomat
    val dodaj_prehod : stanje -> char -> stanje -> avtomat -> avtomat
    val pomozna_funkcija_za_projekcijo_morebitnega_tretjega :
      ('a * 'b * 'c) option -> 'c option
    val preberi_znak : avtomat -> stanje -> char -> stanje option
  end


In [16]:
let enke_1mod3 =
  let open Avtomat in
  let q0 = stanje "q0" and q1 = stanje "q1" and q2 = stanje "q2" in
  prazen_avtomat q0
  |> dodaj_sprejemno_stanje q1
  |> dodaj_nesprejemno_stanje q2
  |> dodaj_prehod q0 '0' q0
  |> dodaj_prehod q1 '0' q1
  |> dodaj_prehod q2 '0' q2
  |> dodaj_prehod q0 '1' q1
  |> dodaj_prehod q1 '1' q2
  |> dodaj_prehod q2 '1' q0

val enke_1mod3 : Avtomat.avtomat =
  {Avtomat.stanja =
    [{Avtomat.oznaka = "q2"}; {Avtomat.oznaka = "q1"};
     {Avtomat.oznaka = "q0"}];
   zacetno = {Avtomat.oznaka = "q0"};
   sprejemna_stanja = [{Avtomat.oznaka = "q1"}];
   prehodi =
    [({Avtomat.oznaka = "q2"}, '1', {Avtomat.oznaka = "q0"});
     ({Avtomat.oznaka = "q1"}, '1', {Avtomat.oznaka = "q2"});
     ({Avtomat.oznaka = "q0"}, '1', {Avtomat.oznaka = "q1"});
     ({Avtomat.oznaka = "q2"}, '0', {Avtomat.oznaka = "q2"});
     ({Avtomat.oznaka = "q1"}, '0', {Avtomat.oznaka = "q1"});
     ({Avtomat.oznaka = "q0"}, '0', {Avtomat.oznaka = "q0"})]}


In [18]:
let niz_ima_pravo_stevilo_enk niz =
  let open Avtomat in
  let avtomat = enke_1mod3 in
  let znaki = List.init (String.length niz) (String.get niz) in
  let aux acc znak =
      match acc with None -> None | Some q -> preberi_znak avtomat q znak
  in
  let koncno_stanje = List.fold_left aux (Some avtomat.zacetno) znaki in
  match koncno_stanje with
  | None -> false
  | Some koncno_stanje -> List.mem koncno_stanje avtomat.sprejemna_stanja

val niz_ima_pravo_stevilo_enk : string -> bool = <fun>


## Signature

In [48]:
module type AVTOMAT = sig
  type stanje
  type avtomat
  val stanje : string -> stanje
  val prazen_avtomat : stanje -> avtomat
  val dodaj_nesprejemno_stanje : stanje -> avtomat -> avtomat
  val dodaj_sprejemno_stanje : stanje -> avtomat -> avtomat
  val dodaj_prehod : stanje -> char -> stanje -> avtomat -> avtomat
  val preberi_znak : avtomat -> stanje -> char -> stanje option
  val zacetno_stanje : avtomat -> stanje
  val je_sprejemno_stanje : avtomat -> stanje -> bool
end

module type AVTOMAT =
  sig
    type stanje
    type avtomat
    val stanje : string -> stanje
    val prazen_avtomat : stanje -> avtomat
    val dodaj_nesprejemno_stanje : stanje -> avtomat -> avtomat
    val dodaj_sprejemno_stanje : stanje -> avtomat -> avtomat
    val dodaj_prehod : stanje -> char -> stanje -> avtomat -> avtomat
    val preberi_znak : avtomat -> stanje -> char -> stanje option
    val zacetno_stanje : avtomat -> stanje
    val je_sprejemno_stanje : avtomat -> stanje -> bool
  end


In [60]:
module Avtomat : AVTOMAT = struct

    type stanje = { oznaka : string }

    type avtomat = {
        stanja : stanje array;
        zacetno_stanje : int;
        sprejemna_stanja : stanje list;
        prehodi : (stanje * char * stanje) list;
    }

    let stanje oznaka = { oznaka }

    let prazen_avtomat zacetno_stanje =
    {
        stanja = [| zacetno_stanje |];
        zacetno_stanje = 0;
        sprejemna_stanja = [];
        prehodi = [];
    }

    let dodaj_v_tabelo x tabela =
        Array.concat [tabela; [|x|]]

    let dodaj_nesprejemno_stanje stanje avtomat =
    { avtomat with stanja = dodaj_v_tabelo stanje avtomat.stanja }

    let dodaj_sprejemno_stanje stanje avtomat =
    {
        avtomat with
        stanja = dodaj_v_tabelo stanje avtomat.stanja;
        sprejemna_stanja = stanje :: avtomat.sprejemna_stanja;
    }

    let dodaj_prehod stanje1 znak stanje2 avtomat =
    { avtomat with prehodi = (stanje1, znak, stanje2) :: avtomat.prehodi }

    let pomozna_funkcija_za_projekcijo_morebitnega_tretjega = function
    | None -> None
    | Some (_, _, q') -> Some q'

    let preberi_znak avtomat q znak =
        avtomat.prehodi
        |> List.find_opt (fun (q1, znak', _q2) -> q1 = q && znak = znak')
        |> pomozna_funkcija_za_projekcijo_morebitnega_tretjega

    let zacetno_stanje avtomat = avtomat.stanja.(avtomat.zacetno_stanje)

    let je_sprejemno_stanje avtomat stanje = List.mem stanje avtomat.sprejemna_stanja

end

module Avtomat : AVTOMAT


In [61]:
Avtomat.preberi_znak

- : Avtomat.avtomat -> Avtomat.stanje -> char -> Avtomat.stanje option =
<fun>


In [62]:
Avtomat.pomozna_funkcija_za_projekcijo_morebitnega_tretjega

error: compile_error

In [63]:
let enke_1mod3 =
  let open Avtomat in
  let q0 = stanje "q0" and q1 = stanje "q1" and q2 = stanje "q2" in
  prazen_avtomat q0
  |> dodaj_sprejemno_stanje q1
  |> dodaj_nesprejemno_stanje q2
  |> dodaj_prehod q0 '0' q0
  |> dodaj_prehod q1 '0' q1
  |> dodaj_prehod q2 '0' q2
  |> dodaj_prehod q0 '1' q1
  |> dodaj_prehod q1 '1' q2
  |> dodaj_prehod q2 '1' q0

val enke_1mod3 : Avtomat.avtomat = <abstr>


In [65]:
let niz_ima_pravo_stevilo_enk niz =
  let open Avtomat in
  let avtomat = enke_1mod3 in
  let znaki = List.init (String.length niz) (String.get niz) in
  let aux acc znak =
      match acc with None -> None | Some q -> preberi_znak avtomat q znak
  in
  let koncno_stanje = List.fold_left aux (Some (zacetno_stanje avtomat)) znaki in
  match koncno_stanje with
  | None -> false
  | Some koncno_stanje -> je_sprejemno_stanje avtomat koncno_stanje

val niz_ima_pravo_stevilo_enk : string -> bool = <fun>
